In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# --- 1. CARGA Y LIMPIEZA DE DATOS (REPETICIÓN DEL PASO ANTERIOR) ---
df = pd.read_csv("Data_Greener_all.csv")
columnas_consumo = ["Ventilation", "Sokets plug", "Lighting", "Other electricity", "Cooling", "Heating"]
num_filas_original = len(df)


In [ ]:
mascara_no_outlier = pd.Series([True] * num_filas_original, index=df.index)

# Aplicar filtrado IQR
for col in columnas_consumo:
    if pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lim_inf = Q1 - 1.5 * IQR
        lim_sup = Q3 + 1.5 * IQR
        mascara_columna = (df[col] >= lim_inf) & (df[col] <= lim_sup)
        mascara_no_outlier = mascara_no_outlier & mascara_columna

df_sin_outliers_final = df[mascara_no_outlier].copy()

In [ ]:
# --- 2. PREPARACIÓN DE DATOS PARA SVR ---

# 2.1 Feature Engineering (Extracción de características de tiempo)
df_sin_outliers_final['Time'] = pd.to_datetime(df_sin_outliers_final['Time'])
df_sin_outliers_final['Hour'] = df_sin_outliers_final['Time'].dt.hour
df_sin_outliers_final['DayOfWeek'] = df_sin_outliers_final['Time'].dt.dayofweek # Lunes=0, Domingo=6
df_sin_outliers_final['Month'] = df_sin_outliers_final['Time'].dt.month

# 2.2 Definir Target (y) y Features (X)
target_column = 'Heating'
feature_columns = [
    'Ventilation', 'Sokets plug', 'Lighting', 'Other electricity', 'Cooling',
    'Hour', 'DayOfWeek', 'Month'
]

X = df_sin_outliers_final[feature_columns]
y = df_sin_outliers_final[target_column]

# 2.3 Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2.4 Scaling (Estandarización de características)
# SVM REQUIERE que los datos de entrada estén escalados.
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel() # Se escala el target para SVR

In [ ]:
# --- 3. ENTRENAMIENTO Y EVALUACIÓN DE SVR ---

# Inicializar y Entrenar el modelo SVR (con kernel RBF)
svr_model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
svr_model.fit(X_train_scaled, y_train_scaled)

# Predicción y Desescalado
y_test_pred_scaled = svr_model.predict(X_test_scaled)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).ravel()

# Evaluación del modelo
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)